In [ ]:
import os

In [ ]:
PATH =  os.getcwd()

In [ ]:
%run -i code/utils.py

# 1. Initialization

In [ ]:
# Prompts user for FDS and fds2ascii executable paths
fds_bin = input('File path to FDS executable: ')
fds2ascii = input('File path to fds2ascii utility in FDS: ')

## 1.1 Initial Region of Interest

In [ ]:
# Reads in elevation file to extract region information
name_of_file = 'Data/chimmey_tops_dem_1m_utm17n.csv'
Mst_original = read_elevation(name_of_file)

In [ ]:
# Coordinates of the Region's Border 
Min_x = 400        
Max_x = 600

Min_y = 0
Max_y = 200

Mst   = Mst_original.copy()
Mst = Mst[(Mst.x > Min_x-1) 
        & (Mst.x < Max_x) 
        & (Mst.y > Min_y-1) 
        & (Mst.y < Max_y)]

In [ ]:
# Defines the Resolution of the Meshes and the obstacles (in Meters)
R = 4                                   # Resolution of Cells
Mst = Mst[(Mst.x%R==0) & (Mst.y%R==0)]  # Filters data 
Ro = 4                                  # Resolution of Obstacles

In [ ]:
# Defines range of elevation
Min_z = math.floor(Mst['Elevation'].min())
Max_z = math.ceil(Mst['Elevation'].max())

## 1.2 Input FDS File Parameters

In [ ]:
# Index for regions that we are simulating
num_region = 1

In [ ]:
# Set time interval
T_begin    = 0.0
DTT        = 60.0
T_end      = T_begin + DTT
DT         = 0.1

# Set Number of meshes in x, y, z directions
Nmx        = 2     
Nmy        = 2     
Nmz        = 2 

PC         = 1         # Predictor-Corrector Status Flag
Location   = [500,100] # Initial Location of the fire
Child      = f"Region{num_region}" # Start of Sequential Domain Decomposition
HRRPUA     = 1500      # Heat Release Rate (HRR)

# File name conventions
foldername = f'chimney'
filename   = f'{Child}.fds'

# Heat Release Rate (HRR) Minimum
hrr_threshold = 10

# Radius of the fire from the center
fire_radius   = 100

# Defining the dataframe with the HRR information
Hrr = Mst[(Mst['x'] < Location[0]+5) & (Mst['x'] > Location[0]-5) & (Mst['y'] < Location[1]+5) & (Mst['y'] > Location[1]-5)]
Hrr = Hrr[[Hrr.columns[0],Hrr.columns[1]]]
Hrr['hrr'] = HRRPUA

# Function to write input FDS file with parameter specified above
write_fds_file(T_begin, T_end, DT, PC, Nmx, Nmy, Nmz, Hrr, Child)

## 1.3 Sets Folders

In [ ]:
FDS_FOLDER = f"FDSFiles/{foldername}" # Location of input FDS file

os.chdir(f'{PATH}/{FDS_FOLDER}')      # Changes directory to run the input FDS file location

## 1.4 Runs FDS

In [ ]:
# User decides to run as a job on a job scheduler or directly through mpiexec command
job_type = input("Are you running with a job scheduler? \nIf so, is it LSF or Slurm?\nIf not, you do not have to specify. ")

In [ ]:
# Sets variables to use in job scripts/commands
num_nodes = 1         # Number of compute nodes to run job on
max_time = '3:00'     # Max time needed to run job
number_of_process = 8 # Number of processes(cores) needed to run the created meshes
omp_threads = 4       # Sets OpenMP Threads per process
jobName = f'FDS_{Child}'

In [ ]:
# Options: LSF, Slurm, or direct command (mpiexec)
if job_type=="LSF" or job_type=="Lsf" or job_type=='lsf':
    USER = os.getlogin()
    create_job_script_lsf(Child, num_nodes, max_time, omp_threads)    # Creates LSF job submission script
    os.system(f'bsub < job_{Child}.bsub') # Submits job script to the LSF job scheduler
    job_id = Get_job_id(['bjobs', '-u', USER])
    #job_id = input("Enter Job ID just started: ") # Gets the job id & waits for it to finish to run the rest of the notebook
    jobs = [job_id]
    wait_on_lsf()
elif job_type=="SLURM" or job_type=="Slurm" or job_type=="slurm":
    USER = os.getlogin()
    create_job_script_slurm(Child, num_nodes, max_time, omp_threads)  # Creates Slurm job submission script
    os.system(f'sbatch job_{Child}.sh')   # Submits job script to the Slurm job Scheduler
    
    job_id = input("Enter Job ID just started: ") # Gets the job id & waits for it to finish to run the rest of the notebook
    jobs = [job_id]
    wait_on_slurm()
else: 
    os.environ['OMP_NUM_THREADS'] = f'{omp_threads}'                  # Sets OpenMP Threads to 4
    os.system(f"mpiexec -n {number_of_process} {fds_bin} {filename}") # Runs FDS using 'mpiexec' command

In [ ]:
remove_leading_space(f'{Child}.smv')  # Corrects .smv in order to work with fds2ascii

# 2 Checkpoint Data Extraction

## 2.1 Parses plot3d files 

In [ ]:
# Reads HRR data in plot3d files
os.chdir(PATH)
Number_of_meshes = Nmx*Nmy*Nmz
reading_hrr(Child, Number_of_meshes)

## 2.2 Extracts & converts HRR data to CSV files

In [ ]:
# Reset current location back to starting location
os.chdir(PATH)

# Converts the CSV Files into Dataframes to be used to extract Heat Release Rate Values
Hrr = pd.DataFrame()
for i in range(1,Number_of_meshes+1):
    temp = pd.read_csv(f"{FDS_FOLDER}/{Child}_{i}.csv")
    temp = temp.drop(temp.index[0])
    temp = temp.reset_index(drop=True)
    Hrr  = Hrr.append(temp)
    Hrr = Hrr.reset_index(drop=True)
Hrr = Hrr[[Hrr.columns[0],Hrr.columns[1],Hrr.columns[2],Hrr.columns[7]]]
Hrr.columns = ['x','y','z','hrr']

file1 = open('FDSFiles/chimney/Region1_hrr.csv', 'r') 
Lines = file1.readlines() 
hrr_value = float(Lines[-1].split(",",2)[1])

In [ ]:
# Cleans the data and sets the data type
Hrr["hrr"] = pd.to_numeric(Hrr["hrr"], downcast="float")
Hrr["x"] = pd.to_numeric(Hrr["x"], downcast="float")
Hrr["y"] = pd.to_numeric(Hrr["y"], downcast="float")
Hrr["z"] = pd.to_numeric(Hrr["z"], downcast="float")
Mst["x"] = pd.to_numeric(Mst["x"], downcast="float")
Mst["y"] = pd.to_numeric(Mst["y"], downcast="float")
Mst.shape

In [ ]:
# Filters HRR values greater than the specified threshold
Hrr = Hrr[Hrr['hrr'] > hrr_threshold]
Hrr['hrr'] = (hrr_value/(4*4*4*Hrr['hrr'].sum()))*Hrr['hrr']
#suma_hrr = Hrr['hrr'].sum()

# Repeat Steps 1.2 - 3 (Loop)

In [ ]:
# Defining the center of the location of the fire
center_x =  math.floor(Hrr["x"].mean())
center_y =  math.floor(Hrr["y"].mean())

In [ ]:
# Update Region Number to next one, if needed
num_region = num_region + 1

# Coordinates of the new region's border
Min_x = center_x - fire_radius
Max_x = center_x + fire_radius
Min_y = center_y - fire_radius
Max_y = center_y + fire_radius
Mst   = Mst_original.copy()
Mst = Mst[(Mst.x > Min_x-Ro) & (Mst.x < Max_x) & (Mst.y > Min_y-Ro) & (Mst.y < Max_y)]
Mst = Mst[(Mst.x%R==0) & (Mst.y%R==0)]

# Range of elevation
Min_z = math.floor(Mst['Elevation'].min())
Max_z = math.ceil(Mst['Elevation'].max())

In [ ]:
# Defines input variables for the next region (with respect to the previous FDS run)
T_begin = T_end
T_end   = T_begin+DTT
Child   = f"Region{num_region}"
foldername = f'chimney'
filename   = f'{Child}.fds'

In [ ]:
# Recreates input FDS file for the next region of interest
restart_fds_file(T_begin, T_end, DT, PC, Nmx, Nmy, Nmz, Hrr, Child)

# Change current location to the input FDS file location
os.chdir(f"{PATH}/{FDS_FOLDER}") 

# Clear the list of jobids
if 'jobs' in locals():
    jobs.clear()

In [ ]:
# Options: LSF, Slurm, or direct command (mpiexec)
if job_type=="LSF" or job_type=="Lsf" or job_type=='lsf':
    create_job_script_lsf(Child, num_nodes, max_time, omp_threads)    # Creates LSF job submission script
    os.system(f'bsub < job_{Child}.bsub') # Submits job script to the LSF job scheduler
    
    job_id = Get_job_id(['bjobs', '-u', USER])
    #job_id = input("Enter Job ID just started: ") # Gets the job id & waits for it to finish to run the rest of the notebook
    jobs = [job_id]
    wait_on_lsf()
elif job_type=="SLURM" or job_type=="Slurm" or job_type=="slurm":
    create_job_script_slurm(Child, num_nodes, max_time, omp_threads)  # Creates Slurm job submission script
    os.system(f'sbatch job_{Child}.sh')   # Submits job script to the Slurm job Scheduler
    
    job_id = input("Enter Job ID just started: ") # Gets the job id & waits for it to finish to run the rest of the notebook
    jobs = [job_id]
    wait_on_slurm()
else: 
    os.environ['OMP_NUM_THREADS'] = f'{omp_threads}'                  # Sets OpenMP Threads to 4
    os.system(f"mpiexec -n {number_of_process} {fds_bin} {filename}") # Runs FDS using 'mpiexec' command